<a href="https://colab.research.google.com/github/don05050505/don05050505/blob/main/cnnCommunity.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# % load_ext tensorboard

import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D

path = "/content/drive/MyDrive/doh_colab/projects"

In [ ]:
class CNN(Sequential):
  def __init__(self, input_shape, num_classes): #input image = 28 x 28 x 1
    super().__init__()

    self.add(Conv2D(32, kernel_size=(5,5),strides=1,
                    padding='same',activation='relu',
                    input_shape=input_shape,
                    kernel_initializer='he_normal'))

    self.add(MaxPooling2D(pool_size=(2,2), strides=(2,2),
                          padding='valid', data_format='channels_last'))

    self.add(Conv2D(64, kernel_size=(5,5), strides=1,
                          padding='same', activation='relu',
                          kernel_initializer='he_normal'))

    self.add(MaxPooling2D(pool_size=(2,2), strides=(2,2),
                          padding='valid', data_format='channels_last'))


    self.add(Flatten())
    self.add(Dense(1024, activation='relu'))
    self.add(tf.keras.layers.Dropout(0.3))

    self.add(Dense(num_classes, activation='softmax'))

    self.compile(optimizer='adam',
                 loss='categorical_crossentropy',
                 metrics=['accuracy'])




In [ ]:
num_of_classes = 10
(XTrain, yTrain), (XTest, yTest) = tf.keras.datasets.mnist.load_data()
XTrain, XTest = XTrain.astype('float32'), XTest.astype('float32')
XTrain, XTest = XTrain.reshape([-1, 28, 28, 1]), XTest.reshape([-1, 28, 28, 1])
XTrain, XTest = XTrain/255., XTest/255.
yTrain = tf.keras.utils.to_categorical(yTrain, num_of_classes)
yTest = tf.keras.utils.to_categorical(yTest, num_of_classes)

In [ ]:
model = CNN((28,28,1), num_of_classes)
model.summary()

In [ ]:

tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=f"{path}/logs")
history = model.fit(XTrain, yTrain, validation_split=(0.2), epochs=1, batch_size=50, callbacks=[tensorboard_callback])

In [ ]:
!pip install tensorboard
%load_ext tensorboard
%tensorboard --logdir f"{path}/logs"

In [ ]:
model.save(f"{path}myModel_mnmist_CNN_tensor2.12.0_v0")

In [ ]:
results = model.evaluate(XTest, yTest, batch_size=100)
model.save(f"{path}myModel_Cifar_CNN_tensor2.12.0_v0")

In [ ]:
input_data = XTest[0]  # Assuming XTest contains your input data
input_data = np.expand_dims(input_data, axis=0)  # Add a batch dimension for a single sample

pred = model.predict(input_data)
label = np.argmax(pred, axis=1)
percs = np.max(pred, axis=1)
print(label, percs)


In [ ]:
y_vloss = history.history['val_loss']  # 검증 세트의 오차
y_loss = history.history['loss']  #  학습 세트의 오차

In [ ]:
x_len = np.arange(len(y_loss))
plt.plot(x_len, y_vloss, marker='.', c='red', label='Test_Set_Loss')
plt.plot(x_len, y_loss, marker='^', c='green', label='Train_Set_Loss')
plt.legend(loc='upper right')
# plt.axis([0, 20, 0, 0.35])
plt.grid()
plt.xlabel('epoch')
plt.ylabel('loss')
save_image('cnnMnist')
plt.show()

In [ ]:
import os

import matplotlib.pyplot as plt

path = '/content/drive/MyDrive/doh_colab/projects'
def save_image(series, flag=True, extension='png', resolution=300):
    if not os.path.isdir(path): os.makedirs(path)
    imagePath = os.path.join(path, f"{series}.{extension}")
    print(f"Save Image: {series}")
    if flag: plt.tight_layout()
    plt.savefig(imagePath, format=extension, dpi=resolution)